In [1]:
import matplotlib
matplotlib.use('TkAgg')  # Set Matplotlib backend to TkAgg (for interactive plotting on some systems)

import numpy as np
import matplotlib.pyplot as plt  # Standard plotting
from matplotlib.animation import FuncAnimation  # Optional: for animations

from shapely.geometry import LineString, Point  # For geometric computations
from shapely.strtree import STRtree  # For spatial indexing of geometries
from collections import defaultdict

import networkx as nx # reconstruct network

import random  # For Python’s own RNG (not used yet)

plt.rc("text", usetex=False)  # Use standard text rendering (no LaTeX)
plt.rc("font", family = "serif")  # Set font to serif
plt.rc("figure", figsize=(10,8))  # Default figure size

%config InlineBackend.figure_format = 'retina'  # Jupyter setting for high-res inline plots (won’t work in script mode)

In [2]:
# particle dynamics 

class Particle:
    def __init__(self, x, y, p_run, p_tumble, theta, speed, p_link, L_link_mean, L_link_std):
        self.position = np.array([x, y])  # Current position (in unit square)
        self.p_run = p_run  # Probability of running at each timestep
        self.p_tumble = p_tumble  # Probability of tumbling (direction change)
        self.theta = theta  # Current heading (angle in radians)
        self.speed = speed  # Speed per timestep
        self.p_link = p_link  # Probability of laying a link at each timestep
        self.L_link_mean = L_link_mean  # Mean link length
        self.L_link_std = L_link_std  # Std dev of link length

    def maybe_lay_link(self):
        if np.random.rand() < self.p_link:
            # Sample link length from Gaussian, fold negative to positive
            L_link = np.abs(np.random.normal(self.L_link_mean, self.L_link_std))
            return True, L_link
        return False, 0.0

    def run(self, dt):
        if np.random.rand() < self.p_run:
            displacement = self.speed * dt # how far does the particle run?
            dx = displacement * np.cos(self.theta) # change in x coordinate (w/ orientation)
            dy = displacement * np.sin(self.theta) # change in y coordinate (w/ orientation)
            self.position = (self.position + np.array([dx, dy])) % 1.0  # Enforce periodic boundaries

    def tumble(self):
        if np.random.rand() < self.p_tumble:
            self.theta = np.random.uniform(-np.pi, np.pi)  # Randomly sample new direction

edges = []  # List of edges as ((x1, y1), (x2, y2)) tuples
nodes = []  # List of node positions (redundant in current logic)

def add_link(particle, L_link):
    # Compute the endpoints of the link centered at particle's position and aligned with heading
    dx = 0.5 * L_link * np.cos(particle.theta)
    dy = 0.5 * L_link * np.sin(particle.theta)
    start = (particle.position - np.array([dx, dy])) % 1.0  # Periodic wrap
    end = (particle.position + np.array([dx, dy])) % 1.0 # Periodic wrap
    edges.append((start, end))  # Store edge
    nodes.append(particle.position.copy())  # Currently adds a node at every deposition (may not be desired)

def simulate(N, steps, dt, particle_params):
    global nodes, edges
    nodes, edges = [], []  # Clear previous run data

    # Create N particles with randomized initial positions and angles
    particles = [
        Particle(
            x=np.random.rand(), y=np.random.rand(),  # Random position
            theta=np.random.uniform(-np.pi, np.pi),  # Random direction
            **particle_params  # Fill in other parameters
        )
        for _ in range(N)
    ]

    for step in range(steps):
        for p in particles:
            laid_link, L_link = p.maybe_lay_link()
            if laid_link:
                add_link(p, L_link)  # Add link if randomly chosen
            p.run(dt)  # Move forward (maybe)
            p.tumble()  # Change direction (maybe)

    # Placeholder for optional post-processing (like merging links)
    #edges[:] = merge_collinear_links(edges)

def plot_network(nodes, edges, step=None):
    plt.figure(figsize=(6, 6))
    for start, end in edges:
        if np.linalg.norm(np.array(start) - np.array(end)) > 0.5:
            continue  # Skip links that wrap around the periodic boundary
        x_vals = [start[0], end[0]]
        y_vals = [start[1], end[1]]
        plt.plot(x_vals, y_vals, 'b-', alpha=0.5)  # Draw edge as blue line

    if nodes:
        xs, ys = zip(*nodes)
        plt.scatter(xs, ys, color='red', s=10, label='Nodes')  # Draw nodes as red dots

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().set_aspect('equal')  # Keep square aspect ratio
    plt.title(f"Network at Step {step}" if step is not None else "Network")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [3]:
from shapely.geometry import LineString, Point, MultiPoint, GeometryCollection
from shapely.ops import split
import networkx as nx

def build_network_by_splitting(edges, tol=1e-6):
    """
    Build a network by splitting each deposited link at its true intersections,
    handling Point, MultiPoint, LineString overlaps, and GeometryCollections.
    """
    # 1) Gather raw intersection POINTS for each link index
    intersections = {i: [] for i in range(len(edges))}
    for i, (s1, e1) in enumerate(edges):
        line1 = LineString([s1, e1])
        for j in range(i+1, len(edges)):
            s2, e2 = edges[j]
            line2 = LineString([s2, e2])
            inter = line1.intersection(line2)
            if inter.is_empty:
                continue

            # collect all intersection points
            pts = []
            if isinstance(inter, Point):
                pts = [inter]
            elif inter.geom_type == "MultiPoint":
                pts = list(inter.geoms)
            elif isinstance(inter, LineString):
                coords = list(inter.coords)
                pts = [Point(coords[0]), Point(coords[-1])]
            elif isinstance(inter, GeometryCollection):
                for part in inter.geoms:
                    if isinstance(part, Point):
                        pts.append(part)
                    elif isinstance(part, LineString):
                        c = list(part.coords)
                        pts.extend([Point(c[0]), Point(c[-1])])
            # else: skip any other geometry types

            # record only if truly on both segments
            for p in pts:
                if line1.distance(p) < tol and line2.distance(p) < tol:
                    intersections[i].append(p)
                    intersections[j].append(p)

    # 2) Split each link at its intersection points and add to graph
    G = nx.Graph()
    for i, (start, end) in enumerate(edges):
        line = LineString([start, end])
        pts = intersections[i]
        if not pts:
            continue

        # round and dedupe intersection points
        mp = MultiPoint([(round(p.x,5), round(p.y,5)) for p in pts])
        pieces = split(line, mp)  # returns a GeometryCollection

        # iterate the actual segments in pieces.geoms
        for seg in pieces.geoms:
            if seg.length < tol:
                continue
            a, b = seg.coords[0], seg.coords[-1]
            a = (round(a[0],5), round(a[1],5))
            b = (round(b[0],5), round(b[1],5))
            G.add_node(a)
            G.add_node(b)
            G.add_edge(a, b, weight=seg.length)

    return G

def plot_graph(G):
    pos = {node: node for node in G.nodes()}  # Position = coordinate
    nx.draw(G, pos, node_size=30, node_color='red', edge_color='blue', with_labels=False)
    plt.gca().set_aspect('equal')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [4]:
# 1. Define particle behavior parameters
params = {
    'p_run': 0.5,           # 50% chance of running
    'p_tumble': 0.5,        # 50% chance of tumbling
    'speed': 0.05,          # Distance per time step
    'p_link': 0.2,          # 20% chance of laying a link per step
    'L_link_mean': 0.1,     # Average link length
    'L_link_std': 0.02      # Std deviation of link length
}

# 2. Run the simulation
steps = 100
N = 10
dt = 1.0

simulate(N=N, steps=steps, dt=dt, particle_params=params)

# 3. Optional: Visualize the raw links and deposition points
plot_network(nodes, edges, step=steps)

# 4. Build the network via binned spatial intersection detection
G = build_network_by_splitting(edges)

# 5. Visualize the final network graph
plot_graph(G)

In [8]:
# ── Ground‐truth test suites ─────────────────────────────────────────────────

# Each is a list of ((x1,y1),(x2,y2)) segments
ground_truths = {
    "X_cross": [ # Two diagonals crossing at (0.5, 0.5). Node at (0.5, 0.5), no edges
        ((0.2, 0.2), (0.8, 0.8)),
        ((0.2, 0.8), (0.8, 0.2))
    ],
    "T_cross": [ # T_cross: A horizontal and two vertical segments meeting in a T at (0.5, 0.5). Node at (0.5, 0.5), no edges
        ((0.1, 0.5), (0.9, 0.5)),
        ((0.5, 0.5), (0.5, 0.9)),
        ((0.5, 0.5), (0.5, 0.1))
    ],
    "parallel": [ # Two parallel lines. No nodes or intersections.
        ((0.1, 0.2), (0.9, 0.2)),
        ((0.1, 0.4), (0.9, 0.4))
    ],
    "triangle": [
        ((0.2, 0.2), (0.5, 0.8)), 
         ((0.8, 0.2), (0.2, 0.2)), 
         ((0.8, 0.2), (0.5, 0.8))
    ],
    "square": [
        ((0.2, 0.2), (0.2, 0.8)), 
        ((0.2, 0.8), (0.8, 0.8)), 
        ((0.8, 0.2), (0.2, 0.2)), 
        ((0.8, 0.2), (0.8, 0.8))
    ],
    "grid": [
        ((0.3, 0.3), (0.3, 0.7)), 
        ((0.3, 0.3), (0.7, 0.3)), 
        ((0.3, 0.7), (0.7, 0.7)), 
        ((0.7, 0.3), (0.7, 0.7))
    ],
    "chain": [((0.7, 0.5), (0.3, 0.5))],
}

def run_ground_truth_tests(builder_fn, L_link_mean=0.1):
    """
    builder_fn: function(edges, ...) → networkx.Graph
    """
    for name, edges in ground_truths.items():
        print(f"\n--- Test case: {name} ---")
        G = builder_fn(edges)
        print(" Nodes:", sorted(G.nodes()))
        print(" Edges:", sorted(G.edges()))
        plot_graph(G)  # if you want to see the layout

if __name__ == "__main__":
    # First, sanity‐check on a few tiny cases:
    run_ground_truth_tests(build_network_by_splitting, L_link_mean=params['L_link_mean'])
    # Or, to test the full binned builder:
    # run_ground_truth_tests(build_binned_network, L_link_mean=params['L_link_mean'])
    
    # Now the real simulation:
    simulate(N=N, steps=steps, dt=dt, particle_params=params)
    plot_network(nodes, edges, step=steps)
    G = build_network_by_splitting(edges)
    plot_graph(G)


--- Test case: X_cross ---
 Nodes: [(0.2, 0.2), (0.2, 0.8), (0.5, 0.5), (0.8, 0.2), (0.8, 0.8)]
 Edges: [((0.2, 0.2), (0.5, 0.5)), ((0.2, 0.8), (0.8, 0.2)), ((0.5, 0.5), (0.8, 0.8))]

--- Test case: T_cross ---
 Nodes: [(0.1, 0.5), (0.5, 0.1), (0.5, 0.5), (0.5, 0.9), (0.9, 0.5)]
 Edges: [((0.1, 0.5), (0.5, 0.5)), ((0.5, 0.5), (0.5, 0.1)), ((0.5, 0.5), (0.5, 0.9)), ((0.5, 0.5), (0.9, 0.5))]

--- Test case: parallel ---
 Nodes: []
 Edges: []

--- Test case: triangle ---
 Nodes: [(0.2, 0.2), (0.5, 0.8), (0.8, 0.2)]
 Edges: [((0.2, 0.2), (0.5, 0.8)), ((0.2, 0.2), (0.8, 0.2)), ((0.5, 0.8), (0.8, 0.2))]

--- Test case: square ---
 Nodes: [(0.2, 0.2), (0.2, 0.8), (0.8, 0.2), (0.8, 0.8)]
 Edges: [((0.2, 0.2), (0.2, 0.8)), ((0.2, 0.2), (0.8, 0.2)), ((0.2, 0.8), (0.8, 0.8)), ((0.8, 0.8), (0.8, 0.2))]

--- Test case: grid ---
 Nodes: [(0.3, 0.3), (0.3, 0.7), (0.7, 0.3), (0.7, 0.7)]
 Edges: [((0.3, 0.3), (0.3, 0.7)), ((0.3, 0.3), (0.7, 0.3)), ((0.3, 0.7), (0.7, 0.7)), ((0.7, 0.3), (0.7, 0.7))]

-

In [22]:
def build_binned_network_debug(edges, L_link_mean, tol=1e-6):
    from shapely.geometry import LineString, Point
    from collections import defaultdict
    import networkx as nx
    import numpy as np

    print("🔨 Starting build_binned_network_debug")
    box_size = L_link_mean
    M = int(np.ceil(1.0/box_size))
    print(f"  box_size = {box_size:.3f}, grid M×M = {M}×{M}")

    def get_box_indices(line):
        minx, miny, maxx, maxy = line.bounds
        ix_min, iy_min = int(minx//box_size), int(miny//box_size)
        ix_max, iy_max = int(maxx//box_size), int(maxy//box_size)
        return [(ix,iy) 
                for ix in range(ix_min, ix_max+1) 
                for iy in range(iy_min, iy_max+1)]

    def round_pt(pt):
        return (round(pt.x,5), round(pt.y,5))

    # 1) Build the grid
    line_index = {}
    grid = defaultdict(list)
    print("\n1) Assigning links to boxes:")
    for i,(s,e) in enumerate(edges):
        line = LineString([s,e])
        line_index[i] = line
        boxes = get_box_indices(line)
        print(f"  link {i}: {s}→{e}, bounds {line.bounds}, boxes {boxes}")
        for b in boxes:
            grid[b].append(i)

    # 2) Find intersections
    intersections = defaultdict(set)
    print("\n2) Finding intersections in each box and neighbors:")
    for ix in range(M):
        for iy in range(M):
            # gather all lines in this cell + neighbors
            local = set()
            for dx in (-1,0,1):
                for dy in (-1,0,1):
                    local |= set(grid.get((ix+dx, iy+dy), []))
            if not local: 
                continue
            print(f"  cell ({ix},{iy}) has line IDs {sorted(local)}")
            L = list(local)
            for a in range(len(L)):
                for b in range(a+1, len(L)):
                    id1,id2 = L[a], L[b]
                    l1,l2 = line_index[id1], line_index[id2]
                    inter = l1.intersection(l2)
                    if inter.is_empty:
                        continue
                    print(f"    → Testing intersection link {id1} vs {id2}: {inter}")
                    # handle points and colinear overlaps
                    pts = []
                    if isinstance(inter, Point):
                        pts = [inter]
                    elif inter.geom_type=="LineString":
                        coords = list(inter.coords)
                        pts = [Point(coords[0]), Point(coords[-1])]
                        print(f"      colinear overlap, endpoints: {coords[0]}, {coords[-1]}")
                    else:
                        print("      skipping geom type", inter.geom_type)
                    for ip in pts:
                        rpt = round_pt(ip)
                        if l1.buffer(tol).contains(ip):
                            intersections[id1].add(rpt)
                        if l2.buffer(tol).contains(ip):
                            intersections[id2].add(rpt)

    # 3) Show raw intersection sets
    print("\n3) Raw intersections by link:")
    for idx, pts in intersections.items():
        print(f"  link {idx} got points {pts}")

    # 4) Build the graph
    G = nx.Graph()
    print("\n4) Building graph edges:")
    for idx, pts in intersections.items():
        line = line_index[idx]
        valid = [pt for pt in pts if 0<=line.project(Point(pt))<=line.length]
        valid.sort(key=lambda pt: line.project(Point(pt)))
        print(f"  link {idx}, sorted pts on segment: {valid}")
        for u,v in zip(valid, valid[1:]):
            d = np.linalg.norm(np.array(u)-np.array(v))
            print(f"    → adding edge {u}–{v}, dist {d:.3f}")
            G.add_edge(u, v, weight=d)

    print("\n✔️ Done.")
    return G

In [28]:
# 1. Define particle behavior parameters
params = {
    'p_run': 0.5,           # 50% chance of running
    'p_tumble': 0.5,        # 50% chance of tumbling
    'speed': 0.05,          # Distance per time step
    'p_link': 0.2,          # 20% chance of laying a link per step
    'L_link_mean': 0.1,     # Average link length
    'L_link_std': 0.02      # Std deviation of link length
}

# 2. Run the simulation
steps = 10
N = 10
dt = 1.0

simulate(N=N, steps=steps, dt=dt, particle_params=params)
plot_network(nodes, edges, step=steps)
G = build_binned_network_debug(edges, L_link_mean=0.1)
plot_graph(G)

🔨 Starting build_binned_network_debug
  box_size = 0.100, grid M×M = 10×10

1) Assigning links to boxes:
  link 0: [0.72582497 0.48293671]→[0.78045388 0.41872788], bounds (0.7258249717069787, 0.4187278778592347, 0.7804538777345642, 0.48293670683471235), boxes [(7, 4)]
  link 1: [0.40651648 0.97246246]→[0.46900713 0.900489  ], bounds (0.406516481177524, 0.9004889985255261, 0.46900712657410487, 0.9724624615334664), boxes [(4, 9)]
  link 2: [0.68454216 0.23207329]→[0.5702607  0.30640418], bounds (0.5702607024774161, 0.23207328802589025, 0.6845421586311421, 0.3064041837730507), boxes [(5, 2), (5, 3), (6, 2), (6, 3)]
  link 3: [0.21351612 0.05879369]→[0.21843253 0.99919562], bounds (0.2135161164887537, 0.05879368538061806, 0.2184325294784731, 0.9991956192255095), boxes [(2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9)]
  link 4: [0.32999353 0.56621319]→[0.23642118 0.60491963], bounds (0.2364211840279087, 0.5662131890866734, 0.3299935270182798, 0.604919630678201